# Free Energy Calculations: Methane in Water

This is a tutorial based on [this](http://www.bevanlab.biochem.vt.edu/Pages/Personal/justin/gmx-tutorials/free_energy/index.html) one. We will use BAC2.0

The system contains a single molecule of methane (called "ALAB" in the coordinate file, for the β-carbon of alanine) in a box of 596 TIP3P water molecules.

In [47]:
import bac.simulate.gromacs as gmx
from pathlib import Path

In [48]:
em_steep = gmx.Run(integrator='steep', number_of_steps=5000)
em_steep.name = 'em_steep'
em_steep.coordinates = Path('input_files/methane_water.gro').absolute()
em_steep.topology = Path('input_files/topol.top').absolute()

em_steep.energy_log_output_frequency = 1
em_steep.energy_output_frequency = 100

In [49]:
em_steep.generate_velocities = False

em_steep.minimization_tolerance = 100
em_steep.minimization_step_size = 0.01
em_steep.minimization_correction_steps = 10

In [50]:
nbc = em_steep.non_bonded_controller
nbc.cutoff_scheme = gmx.non_bonded_controller.CutoffSchemeType.verlet
nbc.neighbor_list_update_frequency = 1
nbc.neighbor_list_type = gmx.non_bonded_controller.NeighborListType.grid
nbc.periodic_boundary_condition = gmx.non_bonded_controller.PeriodicBoundaryConditionType.xyz
nbc.cutoff = 1.2

nbc.coulomb_type = gmx.non_bonded_controller.CoulombType.pme
nbc.coulomb_cutoff = 1.2

nbc.van_der_waals_type = gmx.non_bonded_controller.VanDerWaalsType.cutoff
nbc.van_der_waals_modifier = gmx.non_bonded_controller.VanDerWaalsModifierType.potential_shift
nbc.van_der_waals_switch_cutoff = 1.0
nbc.van_der_waals_cutoff = 1.2

nbc.correct_long_range_dispersion = gmx.non_bonded_controller.LongRangeDispersionCorrectionType.energy_and_pressure

nbc.fourier_spacing = 0.12
nbc.pme_order = 6
nbc.ewald_tolerance_coulomb = 1e-06
nbc.ewald_epsilon_surface = 0

In [51]:
fe = gmx.FreeEnergyController()
em_steep.free_energy_controller = fe

fe.initial_lambda_state = 0
fe.delta_lambda = 0
fe.calculate_lambda_neighbors = 1
fe.van_der_waals_lambdas = [f'{x*0.05:.2f}' for x in range(21)] 
fe.coulomb_lambdas = [f'{0:.2f}']*21
fe.bonded_lambdas = [f'{0:.2f}']*21
fe.restraint_lambdas = [f'{0:.2f}']*21
fe.mass_lambdas = [f'{0:.2f}']*21
fe.temperature_lambdas = [f'{0:.2f}']*21

fe.soft_core_alpha = 0.5
fe.soft_core_coulomb = False
fe.soft_core_power = 1
fe.soft_core_sigma = 0.3

fe.couple_molecule_groups = 'Methane'
fe.couple_type_initial_lambda = 'vdw'
fe.couple_type_final_lambda = 'none'
fe.couple_intramolecular = False

fe.output_frequency = 10

In [52]:
em_steep.constraints.continuation = False
em_steep.constraints.type = gmx.ConstraintType.h_bonds
em_steep.constraints.algorithm = gmx.ConstraintAlgorithmType.lincs
em_steep.constraints.lincs_order = 12

Next energy minimization step

In [53]:
em_bfgs = next(em_steep)

In [54]:
em_bfgs.integrator = gmx.Integrator.l_bfgs
em_bfgs.constraints.type = gmx.ConstraintType.none
em_bfgs.name = 'em_bfgs'

In [55]:
nvt = next(em_bfgs)
nvt.name = 'nvt'

In [56]:
nvt.integrator = 'sd'
nvt.delta_time = 0.002
nvt.number_of_steps = 50000
nvt.com_motion_removal_frequency = 100

nvt.generate_velocities = True
nvt.generate_temperature = 300

In [57]:
nvt.temperature_controller.groups = 'system'
nvt.temperature_controller.time = 1
nvt.temperature_controller.temperature = 300

## Simulation and Workflow

In [58]:
from bac.simulate import Workflow, replica, lambda_window

In [59]:
wf = Workflow(resource='MPB', name='MeOH_H2O')

In [60]:
wf.add_simulation(em_steep)

In [61]:
wf.ensembles = [lambda_window(2), replica(2)]

In [62]:
wf.preprocess_simulations()

gmx grompp -f lambda_0/replica_0/em_steep.mdp -c /Users/kristofarkas/Developer/BAC2/docs/examples/gromacs_fe/input_files/methane_water.gro -p /Users/kristofarkas/Developer/BAC2/docs/examples/gromacs_fe/input_files/topol.top -o lambda_0/replica_0/em_steep.tpr -po lambda_0/replica_0/em_steep_generated.mdp 
b'turning H bonds into constraints...\nturning H bonds into constraints...\nAnalysing residue names:\nThere are:     1      Other residues\nThere are:   596      Water residues\nAnalysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...\nCalculating fourier grid dimensions for X Y Z\nUsing a fourier grid of 25x25x25, spacing 0.119 0.119 0.119\nThis run will generate roughly 14 Mb of data\n'
b'                      :-) GROMACS - gmx grompp, 2016.3 (-:\n\n                            GROMACS is written by:\n     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   \n Aldert van Buuren   Rudi van Drunen     Anton Feenstra    Gerrit Groenhof

gmx grompp -f lambda_1/replica_1/em_steep.mdp -c /Users/kristofarkas/Developer/BAC2/docs/examples/gromacs_fe/input_files/methane_water.gro -p /Users/kristofarkas/Developer/BAC2/docs/examples/gromacs_fe/input_files/topol.top -o lambda_1/replica_1/em_steep.tpr -po lambda_1/replica_1/em_steep_generated.mdp 
b'turning H bonds into constraints...\nturning H bonds into constraints...\nAnalysing residue names:\nThere are:     1      Other residues\nThere are:   596      Water residues\nAnalysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...\nCalculating fourier grid dimensions for X Y Z\nUsing a fourier grid of 25x25x25, spacing 0.119 0.119 0.119\nThis run will generate roughly 14 Mb of data\n'
b'                      :-) GROMACS - gmx grompp, 2016.3 (-:\n\n                            GROMACS is written by:\n     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   \n Aldert van Buuren   Rudi van Drunen     Anton Feenstra    Gerrit Groenhof

In [69]:
print('                      :-) GROMACS - gmx grompp, 2016.3 (-:\n\n                            GROMACS is written by:\n     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   \n Aldert van Buuren   Rudi van Drunen     Anton Feenstra    Gerrit Groenhof  \n Christoph Junghans   Anca Hamuraru    Vincent Hindriksen Dimitrios Karkoulis\n    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    \n  Justin A. Lemkul   Magnus Lundborg   Pieter Meulenhoff    Erik Marklund   \n   Teemu Murtola       Szilard Pall       Sander Pronk      Roland Schulz   \n  Alexey Shvetsov     Michael Shirts     Alfons Sijbers     Peter Tieleman  \n  Teemu Virolainen  Christian Wennberg    Maarten Wolf   \n                           and the project leaders:\n        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel\n\nCopyright (c) 1991-2000, University of Groningen, The Netherlands.\nCopyright (c) 2001-2017, The GROMACS development team at\nUppsala University, Stockholm University and\nthe Royal Institute of Technology, Sweden.\ncheck out http://www.gromacs.org for more information.\n\nGROMACS is free software; you can redistribute it and/or modify it\nunder the terms of the GNU Lesser General Public License\nas published by the Free Software Foundation; either version 2.1\nof the License, or (at your option) any later version.\n\nGROMACS:      gmx grompp, version 2016.3\nExecutable:   /usr/local/gromacs/bin/gmx\nData prefix:  /usr/local/gromacs\nWorking dir:  /Users/kristofarkas/Developer/BAC2/docs/examples/gromacs_fe/MeOH_H2O_4198\nCommand line:\n  gmx grompp -f lambda_0/replica_0/em_steep.mdp -c /Users/kristofarkas/Developer/BAC2/docs/examples/gromacs_fe/input_files/methane_water.gro -p /Users/kristofarkas/Developer/BAC2/docs/examples/gromacs_fe/input_files/topol.top -o lambda_0/replica_0/em_steep.tpr -po lambda_0/replica_0/em_steep_generated.mdp\n\n\nNOTE 1 [file lambda_0/replica_0/em_steep.mdp]:\n  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note\n  that with the Verlet scheme, nstlist has no effect on the accuracy of\n  your simulation.\n\nSetting the LD random seed to 1382163550\nGenerated 331705 of the 331705 non-bonded parameter combinations\nGenerating 1-4 interactions: fudge = 0.5\nGenerated 331705 of the 331705 1-4 parameter combinations\nExcluding 3 bonded neighbours molecule type \'Methane\'\nExcluding 2 bonded neighbours molecule type \'SOL\'\nCoupling 1 copies of molecule type \'Methane\'\nRemoving all charge groups because cutoff-scheme=Verlet\nNumber of degrees of freedom in T-Coupling group rest is 3584.00\nEstimate for the relative computational load of the PME mesh part: 0.35\n\nThere was 1 note\n\nGROMACS reminds you: "Kick the Dog and You Will Die" (Magnapop)\n\n'
)


                      :-) GROMACS - gmx grompp, 2016.3 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra    Gerrit Groenhof  
 Christoph Junghans   Anca Hamuraru    Vincent Hindriksen Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul   Magnus Lundborg   Pieter Meulenhoff    Erik Marklund   
   Teemu Murtola       Szilard Pall       Sander Pronk      Roland Schulz   
  Alexey Shvetsov     Michael Shirts     Alfons Sijbers     Peter Tieleman  
  Teemu Virolainen  Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GROMACS development team at
Uppsala University, Stockholm University